# Importing needed packages

In [1]:
import pprint
import re
import unicodedata

import pandas as pd
import requests
from bs4 import BeautifulSoup

In [2]:
pp = pprint.PrettyPrinter(indent=1, width=140, compact=True)

# Parsing main page

Information, that should be parsed:
 - Film ID
 - Russian title
 - Original title
 - Everything in the section "about the film"
 - Kinopoisk average score & the number of ratings
 - IMDB average score & the number of ratings
 - Film synopsis
 - Film critics rating in the world
     - Percentage
     - Number of reviews
     - Star rating
 - Film critics rating in Russia
     - Percentage
     - Number of reviews

## Getting the code for the page with movie

In [3]:
# f = requests.get("https://www.kinopoisk.ru/film/339/")

## Getting the code from the downloaded page

In [99]:
path_front = "./data/indiana_front.html"
with open(path_front, "r", encoding="utf-8") as f:
    indiana_front = f.read()

Passing the code into the `BeautifulSoup` constructor

In [100]:
soup_front = BeautifulSoup(indiana_front, "html.parser")

In [101]:
movie_dict = {}

## Parsing movie's id

In [102]:
movie_id_dict = {}

In [103]:
film_id = soup_front.find_all(
    "link", attrs={"href": re.compile("https://www.kinopoisk.ru/film/")}
)[0]

In [155]:
film_id_ = int(film_id["href"].split("/")[-2])
film_id_

339

In [156]:
movie_id_dict["id"] = film_id_

## Parsing movie's name

In [106]:
movie_name_dict = {}

In [107]:
movie_name = soup_front.find_all("h1", attrs={"class": re.compile("styles_title")})[0]

In [108]:
movie_name_ = movie_name.get_text()
movie_name_ 

'Индиана Джонс: В поисках утраченного ковчега (1981)'

In [109]:
movie_name_original = soup_front.find_all(
    "span", attrs={"class": re.compile("styles_originalTitle")}
)[0]

In [111]:
movie_name_original_ = movie_name_original.get_text()
movie_name_original_

'Raiders of the Lost Ark'

In [113]:
movie_name_dict["Название"] = movie_name_
movie_name_dict["Оргинальное название"] = movie_name_original_

## Parsing information about the movie

In [114]:
movie_info_dict = {}

In [115]:
movie_info_divs = soup_front.find_all(attrs={"data-test-id": "encyclopedic-table"})[0]

In [116]:
values = []
for div in movie_info_divs.find_all(
    "div", attrs={"class": re.compile("styles_value"), "data-tid": re.compile(".*")}
):
    current_value = div.get_text().replace("сборы", "")
    current_value = unicodedata.normalize("NFKD", current_value)
    if "слова" not in current_value:
        values.append(current_value)

In [117]:
titles = []
for div in movie_info_divs.find_all(class_=re.compile("styles_title")):
    current_title = div.get_text()
    titles.append(current_title)

In [118]:
assert len(values) == len(titles)
movie_info_dict = dict(zip(titles, values))
movie_dict = {**movie_name_dict, **movie_info_dict}

In [119]:
movie_dict

{'Название': 'Индиана Джонс: В поисках утраченного ковчега (1981)',
 'Оргинальное название': 'Raiders of the Lost Ark',
 'Год производства': '1981',
 'Страна': 'США',
 'Жанр': 'приключения, боевик, комедия',
 'Слоган': '«Indiana Jones - the new hero from the creators of JAWS and STAR WARS»',
 'Режиссер': 'Стивен Спилберг',
 'Сценарий': 'Лоуренс Кэздан, Джордж Лукас, Филип Кауфман',
 'Продюсер': 'Ховард Дж. Казанян, Джордж Лукас, Фрэнк Маршалл, ...',
 'Оператор': 'Дуглас Слоком',
 'Композитор': 'Джон Уильямс',
 'Художник': 'Норман Рейнольдс, Айвор Лесли Дилли, Дебора Нэдулмэн, ...',
 'Монтаж': 'Майкл Кан, Джордж Лукас',
 'Бюджет': '$18 000 000',
 'Сборы в США': '$212 222 025',
 'Сборы в мире': '+ $141 766 000 = $353 988 025',
 'Зрители': '88.1 млн , 6.4 млн , 4.2 млн , ...',
 'Премьера в мире': '12 июня 1981, ...',
 'Релиз на DVD': '20 ноября 2003, «Premier Digital»',
 'Возраст': '12+',
 'Рейтинг MPAA': 'PGрекомендуется присутствие родителей',
 'Время': '115 мин. / 01:55'}

## Parsing movie's rating

In [120]:
movie_rating_dict = {}

In [121]:
movie_rating_kinopoisk = soup_front.find_all(
    "a", attrs={"class": re.compile("film-rating-value")}
)[0]

In [123]:
movie_rating_kinopoisk_ = movie_rating_kinopoisk.get_text()
movie_rating_kinopoisk_

'8.0'

In [26]:
movie_rating_count_kinopoisk = soup_front.find_all(
    "span", attrs={"class": re.compile("styles_count")}
)[0]

In [124]:
movie_rating_count_kinopoisk_ = movie_rating_count_kinopoisk.get_text()
movie_rating_count_kinopoisk_

'142 538 оценок'

In [125]:
movie_rating_imdb = soup_front.find_all(
    "span", attrs={"class": re.compile("styles_valueSection")}
)[0]

In [126]:
movie_rating_imdb_ = movie_rating_imdb.get_text()
movie_rating_imdb_

'IMDb: 8.40'

In [30]:
movie_rating_count_imdb = soup_front.find_all(
    "div", attrs={"class": re.compile("film-sub-rating")}
)[0].find("span", attrs={"class": re.compile("styles_count")})

In [127]:
movie_rating_count_imdb_ = movie_rating_count_imdb.get_text()
movie_rating_count_imdb_

'938 581 оценка'

In [128]:
movie_rating_dict["Рейтинг кинопоиска"] = movie_rating_kinopoisk_
movie_rating_dict["Количество оценок кинопоиска"] = movie_rating_count_kinopoisk_
movie_rating_dict["Рейтинг IMDb"] = movie_rating_imdb_
movie_rating_dict["Количество оценок IMDb"] = movie_rating_count_imdb_

In [129]:
movie_rating_dict

{'Рейтинг кинопоиска': '8.0',
 'Количество оценок кинопоиска': '142 538 оценок',
 'Рейтинг IMDb': 'IMDb: 8.40',
 'Количество оценок IMDb': '938 581 оценка'}

## Parsing synopsis

In [130]:
movie_synopsis_dict = {}

In [131]:
movie_synopsis = soup_front.find_all(
    "div", attrs={"class": re.compile("styles_filmSynopsis")}
)[0]

In [133]:
movie_synopsis_ = unicodedata.normalize("NFKD", movie_synopsis.get_text())
movie_synopsis_

'Известный археолог и специалист по оккультным наукам доктор Джонс получает опасное задание от правительства США. Он должен отправиться на поиски уникальной реликвии — священного Ковчега. Но Индиана и не подозревает, что аналогичный приказ уже получили тайные агенты Адольфа Гитлера.'

In [134]:
movie_synopsis_dict = {"Синопсис": movie_synopsis_}

## Parsing critics rating

In [135]:
movie_critics_rating_dict = {}

In [136]:
movie_world_critics = soup_front.find_all(
    "div", attrs={"class": re.compile("styles_ratingBar")}
)[0]

In [137]:
movie_world_critics_percentage = movie_world_critics.find_all(
    "span", attrs={"class": re.compile("film-rating-value")}
)[0]

In [139]:
movie_world_critics_percentage_ = movie_world_critics_percentage.get_text()
movie_world_critics_percentage_

'95%'

In [140]:
movie_world_critics_number_of_reviews = movie_world_critics.find_all(
    "span", attrs={"class": re.compile("styles_count")}
)[0]

In [141]:
movie_world_critics_number_of_reviews_ = movie_world_critics_number_of_reviews.get_text()
movie_world_critics_number_of_reviews_

'86 оценок9.3'

In [142]:
movie_world_critics_star_value = movie_world_critics.find_all(
    "span", attrs={"class": re.compile("styles_starValue")}
)[0]

In [143]:
movie_world_critics_star_value_ = movie_world_critics_star_value.get_text()
movie_world_critics_star_value_

'9.3'

In [144]:
def right_strip_trailing(original, trailing):
    trailing_len = len(trailing)
    if original[-trailing_len:] == trailing:
        return original[:-trailing_len]
    else:
        return original
    return original

In [146]:
movie_world_critics_number_of_reviews_ = right_strip_trailing(
    movie_world_critics_number_of_reviews_, movie_world_critics_star_value_
)
movie_world_critics_number_of_reviews_

'86 оценок'

In [147]:
movie_russian_critics = soup_front.find_all(
    "div", attrs={"class": re.compile("styles_ratingBar")}
)[2]

In [148]:
movie_russian_critics_percentage = movie_russian_critics.find_all(
    "span", attrs={"class": re.compile("film-rating-value")}
)[0]

In [149]:
movie_russian_critics_percentage_ = movie_russian_critics_percentage.get_text()
movie_russian_critics_percentage_

'–'

In [150]:
movie_russian_critics_number_of_reviews = movie_russian_critics.find_all(
    "span", attrs={"class": re.compile("styles_count")}
)[0]

In [152]:
movie_russian_critics_number_of_reviews_ = (
    movie_russian_critics_number_of_reviews.get_text()
)
movie_russian_critics_number_of_reviews_

'1 оценка'

In [153]:
movie_critics_rating_dict = {
    "Средняя полярность международных критиков": movie_world_critics_percentage_,
    "Количество рецензий международных критиков": movie_world_critics_number_of_reviews_,
    "Средняя оценка международных критиков": movie_world_critics_star_value_,
    "Средняя полярность российских критиков": movie_russian_critics_percentage_,
    "Количество рецензий российских критиков": movie_russian_critics_number_of_reviews_,
}
movie_critics_rating_dict

{'Средняя полярность международных критиков': '95%',
 'Количество рецензий международных критиков': '86 оценок',
 'Средняя оценка международных критиков': '9.3',
 'Средняя полярность российских критиков': '–',
 'Количество рецензий российских критиков': '1 оценка'}

# Preparing final dict with information about the film

In [157]:
movie_dict = {
    **movie_id_dict,
    **movie_name_dict,
    **movie_info_dict,
    **movie_rating_dict,
    **movie_synopsis_dict,
    **movie_critics_rating_dict,
}
pp.pprint(movie_dict)

{'id': 339,
 'Бюджет': '$18 000 000',
 'Возраст': '12+',
 'Время': '115 мин. / 01:55',
 'Год производства': '1981',
 'Жанр': 'приключения, боевик, комедия',
 'Зрители': '88.1 млн , 6.4 млн , 4.2 млн , ...',
 'Количество оценок IMDb': '938 581 оценка',
 'Количество оценок кинопоиска': '142 538 оценок',
 'Количество рецензий международных критиков': '86 оценок',
 'Количество рецензий российских критиков': '1 оценка',
 'Композитор': 'Джон Уильямс',
 'Монтаж': 'Майкл Кан, Джордж Лукас',
 'Название': 'Индиана Джонс: В поисках утраченного ковчега (1981)',
 'Оператор': 'Дуглас Слоком',
 'Оргинальное название': 'Raiders of the Lost Ark',
 'Премьера в мире': '12 июня 1981, ...',
 'Продюсер': 'Ховард Дж. Казанян, Джордж Лукас, Фрэнк Маршалл, ...',
 'Режиссер': 'Стивен Спилберг',
 'Рейтинг IMDb': 'IMDb: 8.40',
 'Рейтинг MPAA': 'PGрекомендуется присутствие родителей',
 'Рейтинг кинопоиска': '8.0',
 'Релиз на DVD': '20 ноября 2003, «Premier Digital»',
 'Сборы в США': '$212 222 025',
 'Сборы в мир

# Parsing page with reviews

Information, that should be parsed for every review:
 - Username of reviewer
 - Reviewer ID
 - Date and time of review
 - Review sentiment
 - Review subtitle (if present)
 - Review body (preserving the division into paragraphs)
 - Review score (if possible - the last paragraph of the review)
 - The usefulness of review ratio
 - Direct link to the review

## Getting the code for the page with movie

In [55]:
# f = requests.get("https://www.kinopoisk.ru/film/339/reviews/")

## Getting the code from the downloaded page

In [158]:
path_reviews = "./data/indiana_reviews.html"
with open(path_reviews, "r", encoding="utf-8") as f:
    indiana_reviews = f.read()

Passing the code into the `BeautifulSoup` constructor

In [159]:
soup_reviews = BeautifulSoup(indiana_reviews, "html.parser")

In [160]:
reviews_dict = {}

## Parsing movie's id

Parsing movie id so we can match the reviews with the film

In [161]:
reviews_id_dict = {}

In [162]:
film_id = soup_reviews.find_all(
    "link", attrs={"href": re.compile("https://www.kinopoisk.ru/film/")}
)[0]

In [163]:
film_id_ = film_id["href"].split("/")[-3]

In [164]:
reviews_id_dict["id"] = int(film_id_)
reviews_id_dict

{'id': 339}

## Parsing reviews

In [165]:
reviews = soup_reviews.find_all("div", attrs={"class": "reviewItem userReview"})

Let's start with parsing single review and then we will use this pattern to parse all the reviews on the page

In [166]:
review = reviews[0]

Let's look at the content of review 

In [167]:
review

<div class="reviewItem userReview" data-id="3128202" id="_DRAGGABLE_3128202" style="width: 555px; position: relative; padding: 1px 0 40px 0">
<div id="3128202"></div>
<div class="response good" id="div_review_3128202" itemprop="reviews" itemscope="" itemtype="http://schema.org/Review">
<meta content="Знакомство с Индианой Джонс в 2022." itemprop="headline"/>
<div itemprop="author" itemscope="" itemtype="http://schema.org/Person">
<img alt="" data-lazy-src="https://avatars.mds.yandex.net/get-kino-vod-users-avatar/57335/63366919-43-695797.jpg/46x73_bw" itemprop="image" src="https://st.kp.yandex.net/images/spacer.gif" title="" width="46"/>
<div>
<p class="profile_name"><s></s><a data-popup-info="disabled" href="/user/63366919/" itemprop="name">komisar.jeber</a></p>
<ul class="actions">
<li class="first"><a href="/user/63366919/comments/">рецензии</a></li>
<li><a href="/user/63366919/votes/">оценки</a></li>
<li><a href="/user/63366919/friends/">друзья</a></li>
<li><a href="/user/63366919/m

### Parsing username of reviewer

In [168]:
reviewer_username = review.find_all("p", attrs={"class": "profile_name"})[0]

In [169]:
reviewer_username_ = reviewer_username.get_text()
reviewer_username_

'komisar.jeber'

### Parsing id of reviewer

In [170]:
reviewer_id = reviewer_username.find_all("a", href=True)[0]

In [171]:
reviewer_id_ = reviewer_id["href"].split("/")[-2]
reviewer_id_

'63366919'

### Parsing date & time of review

In [172]:
datetime = review.find_all("span", attrs={"class": "date"})[0]

In [178]:
datetime_ = datetime.get_text().replace("|", "").strip()
datetime_ = ' '.join(datetime_.split())
datetime_

'23 февраля 2022 15:18'

### Parsing review sentiment

In [191]:
sentiment = review.find("div", attrs={"class": "response"})

In [199]:
sentiment_ = sentiment.attrs['class'][-1]
sentiment_

'good'

### Parsing review subtitle

In [200]:
subtitle = review.find("p", attrs={"class": "sub_title"})

In [202]:
subtitle_ = subtitle.get_text()
subtitle_

'Знакомство с Индианой Джонс в 2022.'

### Parsing review body

In [203]:
review_body = review.find("span", attrs={"itemprop": "reviewBody"})

In [212]:
review_body_ = review_body.contents
review_body_

['Спустя более 40 лет после выхода данного фильма я впервые его посмотрел. Ранее видел множество фрагментов из него и кучу отсылок. Так каково же это - впервые посмотреть приключения Индианы Джонс в 2022 году?',
 <br/>,
 '\n',
 <br/>,
 '\nДаже спустя 41 год после премьеры, фильм смотрится довольно прилично. Главные его достоинства это экшен и главный герой, хотя искушенному кинолюбителю этого может показаться мало. Фильм смотрится практически на одном дыхании. Экшен изобретательный и снят отлично, а для 81 года просто невообразимо. Его в фильме более чем достаточно, иногда от него даже можно устать, хоть кино и динамичное.',
 <br/>,
 '\n',
 <br/>,
 '\nСам Индиана Джонс в исполнении Харисона Форда получился то что надо, однако на сегодняшний день он выглядит весьма типично и подобных персонажей в кинохронике уже немало.  Все же это не умаляет заслуг создателей и актер в этой роли смотрится превосходно. Кого-то другого на его месте представить сложно.',
 <br/>,
 '\n',
 <br/>,
 '\nОтдельн

I've decided to remove unnecessary elements and add paragraph separator as `<p>`.  
So paragraphs can be easily separated in the future pre-processing.

In [232]:
review_body_final_ = "<p>".join(
    [
        el.replace("\n", "")
        for el in review_body_
        if str(el) not in ["<br/>", "\n"]
    ]
)
review_body_final_

"Спустя более 40 лет после выхода данного фильма я впервые его посмотрел. Ранее видел множество фрагментов из него и кучу отсылок. Так каково же это - впервые посмотреть приключения Индианы Джонс в 2022 году?<p>Даже спустя 41 год после премьеры, фильм смотрится довольно прилично. Главные его достоинства это экшен и главный герой, хотя искушенному кинолюбителю этого может показаться мало. Фильм смотрится практически на одном дыхании. Экшен изобретательный и снят отлично, а для 81 года просто невообразимо. Его в фильме более чем достаточно, иногда от него даже можно устать, хоть кино и динамичное.<p>Сам Индиана Джонс в исполнении Харисона Форда получился то что надо, однако на сегодняшний день он выглядит весьма типично и подобных персонажей в кинохронике уже немало.  Все же это не умаляет заслуг создателей и актер в этой роли смотрится превосходно. Кого-то другого на его месте представить сложно.<p>Отдельно хочется выделить Карен Ален и ее персонажа в частности. Для стандартной девушки 